In [1]:
import pandas as pd
import numpy as np
import os
import utils

In [2]:
features_1 = [
    'sx_x',
    'sx_y',
    'sx_z',
    'sx_roll',
    'sx_pitch',
    'sx_yaw',
    'sx_thumb',
    'sx_forefinger',
    'sx_middle_finger',
    'sx_ring_finger',
    'sx_little_finger',
    'dx_x',
    'dx_y',
    'dx_z',
    'dx_roll',
    'dx_pitch',
    'dx_yaw',
    'dx_thumb',
    'dx_forefinger',
    'dx_middle_finger',
    'dx_ring_finger',
    'dx_little_finger'
    ]



In [3]:
path = "./tctodd/"
#path = "../../Desktop/MML Project/tctodd/"
dirs = os.listdir(path=path)
weeks = sorted([i for i in dirs if i != ".DS_Store"])
filenames = sorted(os.listdir(path+weeks[1]))

data = []
labels = dict()
label_cnt = 0

for w in weeks:
    temp_path = path+w+"/"
    filenames = sorted(os.listdir(temp_path))
    for fn in filenames:
        label = fn.split('.')[0][:-2]
        
        if label not in labels:
            labels[label] = label_cnt
            label_cnt += 1
            
        data.append({'label':labels[label], 'time_series':pd.read_csv(temp_path+fn, header=None, sep='\t',).values})
        

In [4]:
df = pd.DataFrame(data, columns=['label', 'time_series'])
print(df['label'].value_counts())
df.head()

0     27
60    27
69    27
68    27
67    27
      ..
29    27
28    27
27    27
26    27
94    27
Name: label, Length: 95, dtype: int64


,label,time_series
0,0,"[[-0.064909, 0.034318, -0.043964, 0.626383, 0...."
1,0,"[[-0.107059, -0.126109, -0.053742, 0.612516, 0..."
2,0,"[[-0.061427, -0.082576, -0.102991, 0.735469, 0..."
3,1,"[[-0.128178, 0.02695, -0.050126, 0.455028, 0.4..."
4,1,"[[-0.143672, -0.144416, -0.047447, 0.660979, 0..."


----------

In [8]:
##MISSING THE PARAMETER SEARCH?
import sklearn as sk
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import itertools
from tqdm import tqdm

seed = 0

X = df['time_series']
y = df['label']

# 10-fold cross validation


#best_combination = ('poly', 1e-05, 5, 23)
n_pcs = 55



##LINEAR KERNEL
params = [['linear'], [0.0001, 0.0005, 0.001, 0.005, 0.01, 1]]
params_comb = list(itertools.product(*params))

##RBF KERNEL
params = [['rbf'], [0.0001, 0.0005, 0.001, 0.005, 0.01, 1], [0.001, 0.01, 1, 2, 5, 10]]
params_comb += list(itertools.product(*params))

##POLYNOMIAL KERNEL
params = [['poly'], [1e-5, 0.0001, 0.0005, 0.001, 0.005, 0.01, 1], [0.001, 0.01, 1, 2, 5, 10], [3, 6, 10, 15, 20, 23, 25, 30], [0, 0.1, 0.5, 1, 5, 10]]
params_comb = list(itertools.product(*params))

best_comb = ''
best_acc = 0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.1, random_state=seed)
X_train_matrix = np.vstack(X_train)
means_train = np.mean(X_train_matrix, axis=0)
vars_train = np.var(X_train_matrix, axis=0)

S, v_list_train = utils.compute_S_matrix(X_train, means_train, vars_train)
_, v_list_test = utils.compute_S_matrix(X_test, means_train, vars_train)
w = utils.compute_weight_vector(S, algorithm=2)
print("Performing PCA...")
K_eros_train_mc, V, eig_vals = utils.perform_PCA(len(X_train), weight_vector=w, v_list=v_list_train)
print("Projecting test data...")
Y, K_eros_test_mc = utils.project_test_data(len(X_train), len(X_test), w, v_list_train, v_list_test, K_eros_train_mc, V)

Performing PCA...
-27.38072860425133
now PSD
K eros mean centered is PSD
K eros mean centered is PSD
Projecting test data...


In [9]:
with open("k_eros_file.txt", "a") as f:
    f.write("SVC\n")
    f.write("K-EROS param search without max_iter\n")    
    f.write(f"seed:{seed}\n")

    for params in tqdm(params_comb, desc='doing parameters search...'):
        if len(params) == 2:
            combination = tuple([params[0], params[1], 1, 3, 0])
        elif len(params) == 3:
            combination = tuple([params[0], params[1], params[2], 3, 0])
        else:
            combination = params
        svc = SVC(kernel=combination[0], C=combination[1], gamma=combination[2], degree=combination[3], coef0=combination[4])#, max_iter=1000)
        princ_components = V[:, :n_pcs]
        svc.fit(princ_components, y_train.values)
        test_princ_components = Y[:, :n_pcs]
        predictions = svc.predict(test_princ_components)
        res = accuracy_score(y_test.values, predictions)
        if res > best_acc:
            best_acc = res
            best_comb = combination
            print(f'Found best combination! {best_comb} w. accuracy of {best_acc}.')
    f.write(f"Best combination:{best_comb}\nwith accuracy:{best_acc}\n")

doing parameters search...:   0%|          | 1/2016 [00:00<16:04,  2.09it/s]

Found best combination! ('poly', 1e-05, 0.001, 3, 0) w. accuracy of 0.20622568093385213.


doing parameters search...:   0%|          | 7/2016 [00:03<15:54,  2.10it/s]

Found best combination! ('poly', 1e-05, 0.001, 6, 0) w. accuracy of 0.2140077821011673.


doing parameters search...:   1%|          | 17/2016 [00:08<15:45,  2.11it/s]

...

doing parameters search...:  11%|█▏        | 229/2016 [01:37<11:20,  2.63it/s]

Found best combination! ('poly', 1e-05, 5, 25, 0) w. accuracy of 0.914396887159533.


doing parameters search...:  12%|█▏        | 235/2016 [01:40<11:16,  2.63it/s]

Found best combination! ('poly', 1e-05, 5, 30, 0) w. accuracy of 0.9299610894941635.


doing parameters search...: 100%|██████████| 2016/2016 [13:18<00:00,  2.53it/s]


In [10]:
print(best_comb)

('poly', 1e-05, 5, 30, 0)


In [14]:
#for the best params combination il valore dell accuracy per 10Kfold
with open("k_eros_file.txt", "a") as f:
    skf = sk.model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    mean_accuracy = 0
    n_princ_cs = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
    
    f.write(f"Number of principal components:{n_princ_cs}\n")
    
    res_per_pc = np.zeros(len(n_princ_cs))
    for train_index, test_index in skf.split(X, y):
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]
        
        X_train_matrix = np.vstack(X_train)
        means_train = np.mean(X_train_matrix, axis=0)
        vars_train = np.var(X_train_matrix, axis=0)

        S, v_list_train = utils.compute_S_matrix(X_train, means_train, vars_train)
        _, v_list_test = utils.compute_S_matrix(X_test, means_train, vars_train)
        w = utils.compute_weight_vector(S, algorithm=2)
        K_eros_train_mc, V, eig_vals = utils.perform_PCA(len(X_train), weight_vector=w, v_list=v_list_train)
        Y, K_eros_test_mc = utils.project_test_data(len(X_train), len(X_test), w, v_list_train, v_list_test, K_eros_train_mc, V)
        svc = SVC(kernel=best_comb[0], C=best_comb[1], gamma=best_comb[2], degree=best_comb[3], coef0=best_comb[4])
        for i, n_pc in enumerate(n_princ_cs):
            princ_components = V[:, :n_pc]
            svc.fit(princ_components, y_train.values)
            test_princ_components = Y[:, :n_pc]
            predictions = svc.predict(test_princ_components)
            res = accuracy_score(y_test.values, predictions)
            print(f"result of using {n_pc} components: {res:.2f}")
            res_per_pc[i] += res
    print(f'Reached an accuracy of {res_per_pc[-1]/10}.')
    svc_res_per_pc = res_per_pc/10
    f.write(f"SVC results per number of principal components:{svc_res_per_pc}\n")


-27.699317411486145
now PSD
K eros mean centered is PSD
K eros mean centered is PSD
result of using 1 components: 0.00
result of using 5 components: 0.20
result of using 10 components: 0.54
result of using 15 components: 0.56
result of using 20 components: 0.70
result of using 25 components: 0.75
result of using 30 components: 0.81
result of using 35 components: 0.82
result of using 40 components: 0.85
result of using 45 components: 0.86
result of using 50 components: 0.88
result of using 55 components: 0.89
-27.965133013384865
now PSD
K eros mean centered is PSD
K eros mean centered is PSD
result of using 1 components: 0.01
result of using 5 components: 0.19
result of using 10 components: 0.50
result of using 15 components: 0.54
result of using 20 components: 0.66
result of using 25 components: 0.71
result of using 30 components: 0.82
result of using 35 components: 0.82
result of using 40 components: 0.88
result of using 45 components: 0.88
result of using 50 components: 0.91
result o

In [15]:
with open("k_eros_file.txt", "a") as f:
    f.write("--------------------\nRF\n")

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
criterion = ['gini', 'entropy']
n_estimators = [10, 25, 50, 100, 200]
max_depth = [10, 20, 30, 50, 100]
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
params_grid = {
    'criterion': criterion,
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
}
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.1, random_state=seed)
best_comb = ''
best_acc = 0
X_train_matrix = np.vstack(X_train)
means_train = np.mean(X_train_matrix, axis=0)
vars_train = np.var(X_train_matrix, axis=0)


In [26]:
S, v_list_train = utils.compute_S_matrix(X_train, means_train, vars_train)
_, v_list_test = utils.compute_S_matrix(X_test, means_train, vars_train)
w = utils.compute_weight_vector(S, algorithm=2)
K_eros_train_mc, V, eig_vals = utils.perform_PCA(len(X_train), weight_vector=w, v_list=v_list_train)
Y, K_eros_test_mc = utils.project_test_data(len(X_train), len(X_test), w, v_list_train, v_list_test, K_eros_train_mc, V)

-27.38072860425133
now PSD
K eros mean centered is PSD
K eros mean centered is PSD


In [27]:
best_comb = ''
best_acc = 0
for params in tqdm( list(ParameterGrid(params_grid)), desc="Running parameter grid..." ):
    rf = RandomForestClassifier(**params)
    princ_components = V[:, :n_pcs]
    rf.fit(princ_components, y_train.values)
    test_princ_components = Y[:, :n_pcs]
    predictions = rf.predict(test_princ_components)
    res = accuracy_score(y_test.values, predictions)
    if res > best_acc:
        best_acc = res
        best_comb = params
        print(f'Found best combination! {best_comb} w. accuracy of {best_acc}.')
    
with open("k_eros_file.txt", "a") as f:
    f.write(f"Found best combination: {best_comb}, with accuracy: {best_acc}")

Running parameter grid...:   0%|          | 1/450 [00:00<02:02,  3.68it/s]

Found best combination! {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10} w. accuracy of 0.07003891050583658.


Running parameter grid...:   0%|          | 2/450 [00:00<03:26,  2.17it/s]

Found best combination! {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 25} w. accuracy of 0.13618677042801555.


Running parameter grid...:   1%|          | 3/450 [00:01<05:28,  1.36it/s]

...

Running parameter grid...:  57%|█████▋    | 255/450 [09:51<21:06,  6.49s/it]

Found best combination! {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 200} w. accuracy of 0.6498054474708171.


Running parameter grid...:  72%|███████▏  | 325/450 [15:03<12:28,  5.99s/it]

Found best combination! {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 200} w. accuracy of 0.6614785992217899.


Running parameter grid...: 100%|██████████| 450/450 [23:34<00:00,  3.14s/it]


In [28]:
best_comb

{'criterion': 'entropy',
 'max_depth': 30,
 'min_samples_leaf': 1,
 'min_samples_split': 6,
 'n_estimators': 200}

In [30]:
#for the best params combination il valore dell accuracy per 10Kfold
with open("k_eros_file.txt", "a") as f:
    skf = sk.model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    n_princ_cs = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
    res_per_pc = np.zeros(len(n_princ_cs))
    for train_index, test_index in tqdm(skf.split(X, y), desc="Performing stratified 10-fold"):
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]
        X_train_matrix = np.vstack(X_train)
        means_train = np.mean(X_train_matrix, axis=0)
        vars_train = np.var(X_train_matrix, axis=0)

        S, v_list_train = utils.compute_S_matrix(X_train, means_train, vars_train)
        _, v_list_test = utils.compute_S_matrix(X_test, means_train, vars_train)
        w = utils.compute_weight_vector(S, algorithm=2)
        K_eros_train_mc, V, eig_vals = utils.perform_PCA(len(X_train), weight_vector=w, v_list=v_list_train)
        Y, K_eros_test_mc = utils.project_test_data(len(X_train), len(X_test), w, v_list_train, v_list_test, K_eros_train_mc, V)
        rf = RandomForestClassifier(**best_comb)
        for i, n_pc in enumerate(n_princ_cs):
            princ_components = V[:, :n_pc]
            rf.fit(princ_components, y_train.values)
            test_princ_components = Y[:, :n_pc]
            predictions = rf.predict(test_princ_components)
            res = accuracy_score(y_test.values, predictions)
            print(f"result of using {n_pc} components: {res:.2f}")
            res_per_pc[i] += res

    print(f'Reached an accuracy of {res_per_pc[-1]/10}.')
    rf_res_per_pc = res_per_pc/10
    f.write(f"Number of principal components:{n_princ_cs}\n")
    f.write(f"RF results per number of principal components:{rf_res_per_pc}\n")



Performing stratified 10-fold: 0it [00:00, ?it/s]

-27.699317411486145
now PSD
K eros mean centered is PSD
K eros mean centered is PSD
result of using 1 components: 0.02
result of using 5 components: 0.09
result of using 10 components: 0.33
result of using 15 components: 0.34
result of using 20 components: 0.46
result of using 25 components: 0.53
result of using 30 components: 0.54
result of using 35 components: 0.59
result of using 40 components: 0.58
result of using 45 components: 0.59
result of using 50 components: 0.63


Performing stratified 10-fold: 1it [04:39, 279.50s/it]

result of using 55 components: 0.60
-27.965133013384865
now PSD
K eros mean centered is PSD
K eros mean centered is PSD
result of using 1 components: 0.02
result of using 5 components: 0.09
result of using 10 components: 0.30
result of using 15 components: 0.37
result of using 20 components: 0.42
result of using 25 components: 0.49
result of using 30 components: 0.56
result of using 35 components: 0.56
result of using 40 components: 0.62
result of using 45 components: 0.58
result of using 50 components: 0.63


Performing stratified 10-fold: 2it [09:30, 286.35s/it]

...

result of using 55 components: 0.64
-27.730006664333168
now PSD
K eros mean centered is not PSD
K eros mean centered is PSD
result of using 1 components: 0.02
result of using 5 components: 0.11
result of using 10 components: 0.37
result of using 15 components: 0.36
result of using 20 components: 0.41
result of using 25 components: 0.45
result of using 30 components: 0.57
result of using 35 components: 0.58
result of using 40 components: 0.58
result of using 45 components: 0.64
result of using 50 components: 0.70


Performing stratified 10-fold: 9it [44:21, 300.87s/it]

result of using 55 components: 0.65
-27.69289686113509
now PSD
K eros mean centered is not PSD
K eros mean centered is PSD
result of using 1 components: 0.02
result of using 5 components: 0.09
result of using 10 components: 0.36
result of using 15 components: 0.33
result of using 20 components: 0.40
result of using 25 components: 0.50
result of using 30 components: 0.55
result of using 35 components: 0.57
result of using 40 components: 0.58
result of using 45 components: 0.66
result of using 50 components: 0.67


Performing stratified 10-fold: 10it [49:25, 296.52s/it]

result of using 55 components: 0.62
Reached an accuracy of 0.6269136065175097.


In [32]:
with open("k_eros_file.txt", "a") as f:
    f.write(f"RF results per number of principal components:{rf_res_per_pc}\n")


print('SVC accuracy x n principal components: ', svc_res_per_pc)
print('RF accuracy x n principal components: ', rf_res_per_pc)

SVC accuracy x n principal components:  [0.00389409 0.21480575 0.55088765 0.59844662 0.7009834  0.7376292
 0.8202669  0.84639622 0.87173517 0.87603204 0.89552073 0.89823839]
RF accuracy x n principal components:  [0.02105119 0.10642479 0.33139135 0.38007812 0.42842899 0.46938382
 0.54814719 0.56881384 0.59258876 0.61056815 0.6327441  0.62691361]
